In [ ]:
import numpy as np
from prose import FITSImage, Sequence, blocks

# reference is middle image
ref = FITSImage(sciences[len(sciences)//2])

calibration = Sequence([
    blocks.Calibration(darks=darks, bias=bias, flats=flats),
    blocks.Trim(),
    blocks.PointSourceDetection(n=20), # stars detection
    blocks.Cutouts(21),                # stars cutouts
    blocks.MedianEPSF(),               # building EPSF
    blocks.psf.Moffat2D(),             # modeling EPSF
])

calibration.run(ref, show_progress=False)
radii = np.linspace(0.2, 8, 30)
calibration[2].n = 15  # only 15 stars for alignment

photometry = Sequence([
    *calibration,                           # calibration
    blocks.AlignReferenceSources(ref),      # alignment
    blocks.CentroidQuadratic(),             # centroiding
    blocks.AperturePhotometry(radii=radii), # aperture photometry
    blocks.AnnulusBackground(),
    blocks.GetFluxes("fwhm","keyword:AIRMASS")]
)

photometry.run(sciences)

In [ ]:
from prose import FITSImage, Sequence, blocks

image = FITSImage("2002MS4_Artemis_clear_2x2bin-0525.fit")

sequence = Sequence([
    blocks.PointSourceDetection(),  # stars detection
    blocks.Cutouts(21),             # cutouts extraction
    blocks.MedianEPSF(),            # PSF building
    blocks.Moffat2D(),              # PSF modeling
])

sequence.run([image])

# plotting the detected stars
image.show()